# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [36]:
from azureml.core import Workspace, Environment, Experiment, ScriptRunConfig
from azureml.core.dataset import Dataset
from azureml.core.model import Model
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.sklearn import SKLearn
from azureml.train.estimator import Estimator
from azureml.widgets import RunDetails

## Dataset

### Overview
I chose the Titanic Dataset from [Kaggle Titanic Challenge](https://www.kaggle.com/c/titanic). 
In the dataset we have information for 871 passengers were more likely to survive than others. 

In [37]:
ws = Workspace.from_config()
experiment_name = 'TitanicData-hyperparam-experiment'

experiment=Experiment(ws, experiment_name)


# Get the data of Kaggle Titanic Dataset
key = "TitanicData2"
description_text = "Kaggle Titanic Challenge dataset with some changes made by myself"
found = False

if key in ws.datasets.keys(): 
    found = True
    dataset = ws.datasets[key] 

if not found:
    # Create AML Dataset and register it into Workspace
    example_data = 'https://raw.githubusercontent.com/clasimoes/nd00333-capstone/master/titanic_data/full_capstone.csv'
    dataset = Dataset.Tabular.from_delimited_files(example_data)
    #Register Dataset in Workspace
    dataset = dataset.register(workspace=ws,
                               name=key,
                               description=description_text)


df = dataset.to_pandas_dataframe()
df.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare,Q,S,male
count,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.066409,0.523008,0.381594,32.204208,0.086420,0.725028,0.647587
std,257.353842,0.486592,0.836071,13.244532,1.102743,0.806057,49.693429,0.281141,0.446751,0.477990
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,22.000000,0.000000,0.000000,7.910400,0.000000,0.000000,0.000000
50%,446.000000,0.000000,3.000000,26.000000,0.000000,0.000000,14.454200,0.000000,1.000000,1.000000
75%,668.500000,1.000000,3.000000,37.000000,1.000000,0.000000,31.000000,0.000000,1.000000,1.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200,1.000000,1.000000,1.000000


In [38]:
# from azureml.core import Workspace, Dataset
df['Survived'] = df['Survived'].astype(int)
updated_dataset = Dataset.Tabular.register_pandas_dataframe(
    dataframe=df,
    target=(ws.get_default_datastore(), 'TitanicData2'),
    name='TitanicData2',
    show_progress=True
)

Validating arguments.
Arguments validated.
Successfully obtained datastore reference and path.
Uploading file to TitanicData2/b6f0eb75-3fb7-49de-9c90-2fe911912adb/
Successfully uploaded file to datastore.
Creating and registering a new dataset.
Successfully created and registered a new dataset.


### Create a compute cluster

We create here a compute cluster to run the experiment. In this cluster, we provise 2-10 machines with the "STANDARD_DS12_V2" configuration.

In [39]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "cluster-capstone"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS3_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=10)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 2, timeout_in_minutes = 10)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Hyperdrive Configuration

In this context, we have harnessed a Logistic Regression model from the SKLearn framework to undertake binary classification, specifically predicting the survival outcomes of passengers in the Titanic shipwreck.

The crux of our endeavor lies in the utilization of Hyperdrive, a hyperparameter optimization technique. This optimization primarily centers around two pivotal algorithmic hyperparameters:

1. "C": This parameter embodies the inverse of regularization strength and plays a vital role in regulating overfitting or underfitting.

2. "max_iter": This parameter dictates the maximum number of iterations that optimization solvers undergo to achieve convergence, a critical aspect for ensuring model stability.

Distinct combinations of these hyperparameters engender disparate model instances, thereby exerting a pronounced impact on model performance. Our overarching objective is to curate models that maximize predictive accuracy.

To facilitate this hyperparameter exploration, we have employed Bayesian Sampling, a sophisticated technique that employs probabilistic models to iteratively select hyperparameter values. This approach allows for an intelligent exploration of the hyperparameter space, leading to more efficient optimization.

Specifically, we leverage Bayesian Sampling to probabilistically determine values for "C" within a continuous range spanning [0.001, 1.0]. For "max_iter," Bayesian Sampling assists in selecting values from a predefined set of three options: 1000, 10000, and 100000.

Additionally, we have instituted an early termination policy to expedite the optimization process by terminating underperforming model runs. The Bandit Policy, configured with a factor of 0.1, dictates that any run failing to meet a predefined slack factor with respect to the evaluation metric (in this case, "Accuracy") compared to the best-performing run will be promptly terminated. This strategic choice serves to conserve computational resources and time.

In summary, the Hyperdrive configuration encompasses a comprehensive set of critical settings. These include the specification of a training script for data ingestion and Logistic Regression model training via SKLearn, the identification of a dedicated compute target (as previously defined), the delineation of an environment specification outlined in a conda_dependencies YAML file, the utilization of Bayesian Sampling for parameter optimization, the formulation of an early termination policy, the definition of the targeted metric for optimization and the strategy employed for its enhancement, and the specification of the total number of runs to be executed within the Hyperdrive optimization framework.

In [40]:
# Create dependencies file for the train script
import os
import shutil

project_folder = './sklearn-titanic-hyperdrive'
os.makedirs(project_folder, exist_ok=True)
shutil.copy('train_sklearn.py', project_folder)
# shutil.copy('config.json', project_folder)
            
sklearn_env = Environment.from_conda_specification(name = 'sklearn-env', file_path = './conda_dependencies.yml')

In [44]:
# Create an early termination policy. 
early_termination_policy = BanditPolicy(slack_factor = 0.1)

# Create the different params that you will be using during training
param_sampling = RandomParameterSampling( {
    '--C': uniform(0.001, 1.0),
    '--max_iter': choice(100, 1000, 10000)
} )

# Create your estimator and hyperdrive config
# src = ScriptRunConfig(source_directory=project_folder,
#                       script='train_sklearn.py',
#                       compute_target=compute_target,
#                       environment=sklearn_env)
estimator = SKLearn(source_directory=project_folder, compute_target=compute_target, entry_script='train_sklearn.py')


hyperdrive_config = HyperDriveConfig(estimator=estimator,
                                     hyperparameter_sampling=param_sampling,
                                     policy=early_termination_policy,
                                     primary_metric_name="Accuracy",
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=100)

'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.


In [45]:
# Submit the experiment
hyperdrive_run = experiment.submit(hyperdrive_config)

## Run Details

Here we have `RunDetails` widget to show the different experiments.

In [46]:
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

## Best Model

Here get the best model from the automl experiments and display all the arguments of the model and its id.

In [47]:
best_run = hyperdrive_run.get_best_run_by_primary_metric()
print(best_run)

Run(Experiment: TitanicData-hyperparam-experiment,
Id: HD_6e709290-5f27-486d-8d37-3f6c712ddc94_17,
Type: azureml.scriptrun,
Status: Completed)


In [48]:
print(best_run)

Run(Experiment: TitanicData-hyperparam-experiment,
Id: HD_6e709290-5f27-486d-8d37-3f6c712ddc94_17,
Type: azureml.scriptrun,
Status: Completed)


In [49]:
print(best_run.get_details()['runId'])

HD_6e709290-5f27-486d-8d37-3f6c712ddc94_17


In [50]:
print(best_run.get_details()['runDefinition']['arguments'])

['--C', '0.9557493405665222', '--max_iter', '1000']


In [51]:
# Model Metrics
print(best_run.get_metrics())

{'Regularization Strength': 0.9557493405665222, 'Max iterations': 1000, 'Accuracy': 0.8565022421524664}


In [52]:
best_run.get_details()

{'runId': 'HD_6e709290-5f27-486d-8d37-3f6c712ddc94_17',
 'target': 'cluster-capstone',
 'status': 'Completed',
 'startTimeUtc': '2023-10-08T03:03:08.258234Z',
 'endTimeUtc': '2023-10-08T03:03:21.790958Z',
 'services': {},
 'properties': {'_azureml.ComputeTargetType': 'amlctrain',
  'ContentSnapshotId': '0b8da89f-f3fe-4297-af7e-0e506694bd6e',
  'ProcessInfoFile': 'azureml-logs/process_info.json',
  'ProcessStatusFile': 'azureml-logs/process_status.json'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'script': 'train_sklearn.py',
  'command': '',
  'useAbsolutePath': False,
  'arguments': ['--C', '0.9557493405665222', '--max_iter', '1000'],
  'sourceDirectoryDataStore': None,
  'framework': 'Python',
  'communicator': 'None',
  'target': 'cluster-capstone',
  'dataReferences': {},
  'data': {},
  'outputData': {},
  'datacaches': [],
  'jobName': None,
  'maxRunDurationSeconds': None,
  'nodeCount': 1,
  'instanceTypes': [],
  'priority': None,
  'credentialPassthrough

In [53]:
print(best_run.get_file_names())

['outputs/model.joblib', 'system_logs/cs_capability/cs-capability.log', 'system_logs/hosttools_capability/hosttools-capability.log', 'system_logs/lifecycler/execution-wrapper.log', 'system_logs/lifecycler/lifecycler.log', 'system_logs/metrics_capability/metrics-capability.log', 'system_logs/snapshot_capability/snapshot-capability.log', 'user_logs/std_log.txt']


In [54]:
# Save the best model 
best_run.download_file('outputs/model.joblib', 'sklearn-titanic/model.joblib')

In [55]:
# Register the best model
model = best_run.register_model(model_name='sklearn-titanic',
                                model_path='outputs/model.joblib',
                                model_framework=Model.Framework.SCIKITLEARN)

## Model Deployment

Create an inference config and deploy the model as a web service.

In [84]:
service_name = 'hyper-service-deployment'
# inference_config = InferenceConfig(entry_script='sklearn-titanic/score.py')

service = Model.deploy(ws, service_name, [model])
service.wait_for_deployment(show_output=True)

azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'


Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2023-10-08 04:11:22+00:00 Creating Container Registry if not exists.
2023-10-08 04:11:22+00:00 Registering the environment.
2023-10-08 04:11:23+00:00 Uploading autogenerated assets for no-code-deployment.
2023-10-08 04:11:24+00:00 Use the existing image.
2023-10-08 04:11:24+00:00 Generating deployment configuration.
2023-10-08 04:11:24+00:00 Submitting deployment to compute.
2023-10-08 04:11:29+00:00 Checking the status of deployment hyper-service-deployment..
2023-10-08 04:13:01+00:00 Checking the status of inference endpoint hyper-service-deployment.
Failed


ERROR:azureml.core.webservice.webservice:Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: 0dcd4544-385f-40da-a903-7244e1f16843
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "statusCode": 400,
  "message": "Aci Deployment failed with exception: Your container application crashed. This may be caused by errors in your scoring file's init() function.
	1. Please check the logs for your container instance: hyper-service-deployment. From the AML SDK, you can run print(service.get_logs()) if you have service object to fetch the logs.
	2. You can interactively debug your scoring file locally. Please refer to https://docs.microsoft.com/azure/machine-learning/how-to-debug-visual-studio-code#debug-and-troubleshoot-deployments for more information.
	3. You can also try to run image 25a57bfbd2c74ce0b74d3d00bf800e72.azurecr.io/azureml/azureml_978d97c2345439089c1a81cf8b739c36 locally. Please re

WebserviceException: WebserviceException:
	Message: Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: 0dcd4544-385f-40da-a903-7244e1f16843
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "statusCode": 400,
  "message": "Aci Deployment failed with exception: Your container application crashed. This may be caused by errors in your scoring file's init() function.
	1. Please check the logs for your container instance: hyper-service-deployment. From the AML SDK, you can run print(service.get_logs()) if you have service object to fetch the logs.
	2. You can interactively debug your scoring file locally. Please refer to https://docs.microsoft.com/azure/machine-learning/how-to-debug-visual-studio-code#debug-and-troubleshoot-deployments for more information.
	3. You can also try to run image 25a57bfbd2c74ce0b74d3d00bf800e72.azurecr.io/azureml/azureml_978d97c2345439089c1a81cf8b739c36 locally

In [83]:
service_name = 'hyper-service'
# inference_config = InferenceConfig(entry_script='sklearn-titanic/score.py')

service = Model.deploy(ws, service_name, [model])
service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2023-10-08 03:45:32+00:00 Creating Container Registry if not exists..
2023-10-08 03:55:33+00:00 Generating deployment configuration.
2023-10-08 03:55:34+00:00 Submitting deployment to compute.
2023-10-08 03:55:38+00:00 Checking the status of deployment capstone-automl-service..
2023-10-08 03:57:30+00:00 Checking the status of inference endpoint capstone-automl-service.
Succeeded
ACI service creation operation finished, operation Succeeded


In [71]:
print(service.state)

Healthy


Send a request to the web service you deployed to test it.

In [ ]:
import json

data = {"data":
        [
          {
            "PassengerId": 812,
            "Pclass": 2,
            "Age": 23.0,
            "SibSp": 0,
            "Parch": 0, 
            "Fare": 13.0,
            "Q": 0,
            "S": 1,
            "male": 1
          },
          {
            "PassengerId": 813,
            "Pclass": 1,
            "Age": 35.0,
            "SibSp": 0,
            "Parch": 0, 
            "Fare": 512.3292,
            "Q": 0,
            "S": 0,
            "male": 1
          }
      ]
    }

# Convert to JSON string
input_data = json.dumps(data)

In [ ]:
output = service.run(input_data)

Print the logs of the web service and delete the service

In [ ]:
print(output)

In the cell above, logistic regression returns an array of observations with the prediction probabilities.

The first item refers to passenger 812. The probability of this passenger belonging to the non-survived (class 0) is 65,55%, while the probability of this passenger belonging to the non-survived (class 1) is 34,45%. Thus we conclude that passenger 812 was classified as non-survivant.

Similarly, passenger 813 has been classified as a "Survived" (class 1) with a chance of 92,85%.

In [27]:
service.delete()

In [28]:
print(service.state)

Deleting
